In [31]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine


## this is a _Data_Engineer_interview

In [7]:
df=pd.read_csv("C:/etl/etl_data_interview/candidates.csv",sep=";")
print(df_2008.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

# Analisis Exploratorio De Los Datos
## ¿Qué tipo de dato son las variables del conjunto de datos?

In [21]:
df.dtypes

First_Name                   object
Last_Name                    object
Email                        object
Application_Date             object
Country                      object
YOE                           int64
Seniority                    object
Technology                   object
Code_Challenge_Score          int64
Technical_Interview_Score     int64
dtype: object

## ¿Cuántas variables de cada tipo de dato tenemos en el conjunto de datos?

In [23]:
(
    df
    .dtypes
    .value_counts()
)

object    7
int64     3
dtype: int64

## ¿Cuántas variables y observaciones tenemos en el conjunto de datos?

In [25]:
df.shape

(50000, 10)

## ¿Existen valores nulos explicitos en el conjunto de datos?

In [28]:
(   df
    .isnull()
    .any()
)

First_Name                   False
Last_Name                    False
Email                        False
Application_Date             False
Country                      False
YOE                          False
Seniority                    False
Technology                   False
Code_Challenge_Score         False
Technical_Interview_Score    False
dtype: bool

In [29]:
df.describe(include='all')

,First_Name,Last_Name,Email,Application_Date,Country,YOE,Seniority,Technology,Code_Challenge_Score,Technical_Interview_Score
count,50000,50000,50000,50000,50000,50000.000000,50000,50000,50000.000000,50000.000000
unique,3007,474,49833,1646,244,NaN,7,24,NaN,NaN
top,Sarai,Murazik,fern70@gmail.com,2020-07-07,Malawi,NaN,Intern,Game Development,NaN,NaN
freq,33,138,3,50,242,NaN,7255,3818,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,15.286980,NaN,NaN,4.996400,5.003880
std,NaN,NaN,NaN,NaN,NaN,8.830652,NaN,NaN,3.166896,3.165082
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,2.000000,2.000000
50%,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,5.000000,5.000000
75%,NaN,NaN,NaN,NaN,NaN,23.000000,NaN,NaN,8.000000,8.000000


## Solo las numéricas

In [32]:
df.describe(include=[np.number])

,YOE,Code_Challenge_Score,Technical_Interview_Score
count,50000.000000,50000.000000,50000.000000
mean,15.286980,4.996400,5.003880
std,8.830652,3.166896,3.165082
min,0.000000,0.000000,0.000000
25%,8.000000,2.000000,2.000000
50%,15.000000,5.000000,5.000000
75%,23.000000,8.000000,8.000000
max,30.000000,10.000000,10.000000


## Solo categóricas - 1

In [33]:
df.describe(include=object)

,First_Name,Last_Name,Email,Application_Date,Country,Seniority,Technology
count,50000,50000,50000,50000,50000,50000,50000
unique,3007,474,49833,1646,244,7,24
top,Sarai,Murazik,fern70@gmail.com,2020-07-07,Malawi,Intern,Game Development
freq,33,138,3,50,242,7255,3818


TRANSFORM 

In [9]:
new_Columns_names=["First_Name","Last_Name","Email","Application_Date","Country","YOE","Seniority","Technology","Code_Challenge_Score","Technical_Interview_Score"]
df.columns=new_Columns_names
df

,First_Name,Last_Name,Email,Application_Date,Country,YOE,Seniority,Technology,Code_Challenge_Score,Technical_Interview_Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


CONEXION A LA BASE DE DATOS

In [11]:
# Parámetros de conexión
db_params = {
    "host": "localhost",          # La dirección IP o el nombre de host del contenedor
    "port": "5435",               # El puerto mapeado del contenedor (5435)
    "database": "postgres",       # Nombre de la base de datos (por defecto)
    "user": "postgres",           # Usuario de la base de datos
    "password": "mysecretpass",   # Contraseña del usuario
}

# Intentar conectarse a la base de datos
try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Ejecutar consultas aquí
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Conexión exitosa a PostgreSQL:", version)

    # Cerrar la conexión
    cursor.close()
    connection.close()

except Exception as e:
    print("Error de conexión:", e)

Conexión exitosa a PostgreSQL: ('PostgreSQL 15.3 (Debian 15.3-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [20]:
# Creo que se puede mejorar, llamando solamente al connection
connection= psycopg2.connect(**db_params)
cursor = connection.cursor()

create_table_query = """
    CREATE TABLE IF NOT EXISTS Applicants (
    id serial PRIMARY KEY,
    first_name varchar,
    last_name varchar,
    email varchar,
    application_date date,
    country varchar,
    yoe integer,
    seniority varchar,
    technology varchar,
    code_challenge_score integer,
    technical_interview_score integer
);
"""
cursor.execute(create_table_query)
connection.commit()

consulta = "SELECT * FROM mi_tabla"

cursor.execute(consulta)
#Obtener los resultados de la consulta
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)
cursor.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



('Taylor', 'Ernser', 'marisa22@gmail.com', '2020-11-18', 'Cayman Islands', 25, 'Mid-Level', 'Database Administration', 6, 9)
('Bret', 'Metz', 'leora65@gmail.com', '2019-04-08', 'Singapore', 29, 'Architect', 'Development - FullStack', 0, 5)
('Daisha', 'Wiegand', 'pascale.wilkinson@hotmail.com', '2021-10-15', 'Saint Kitts and Nevis', 17, 'Junior', 'QA Manual', 9, 1)
('Aimee', 'Huels', 'mittie.satterfield43@hotmail.com', '2018-10-07', 'Togo', 4, 'Junior', 'Sales', 5, 8)
('Amina', 'Gislason', 'garrick.nikolaus@gmail.com', '2022-04-07', 'Puerto Rico', 0, 'Intern', 'Salesforce', 4, 7)
('Ian', 'Lubowitz', 'arthur80@yahoo.com', '2018-08-26', 'Iran', 12, 'Architect', 'DevOps', 1, 6)
('Lura', 'VonRueden', 'jerald_hickle84@yahoo.com', '2022-02-04', 'Cameroon', 26, 'Trainee', 'System Administration', 9, 4)
('Elyse', 'Deckow', 'mozelle.gulgowski24@hotmail.com', '2020-05-26', 'Croatia', 7, 'Senior', 'Game Development', 4, 0)
('Hermann', 'Heathcote', 'constance.koch@gmail.com', '2018-08-22', 'Saint B

In [15]:
# Crear una cadena de conexión a la base de datos PostgreSQL
db_connection_string = (
    f"postgresql://{db_params['user']}:{db_params['password']}@"
    f"{db_params['host']}:{db_params['port']}/{db_params['database']}"
)

# Crear una instancia de SQLAlchemy Engine
engine = create_engine(db_connection_string)

# Cargar el DataFrame en la tabla "mi_tabla" en la base de datos
tabla_destino = "Applicants"
df.to_sql(tabla_destino, engine, if_exists="append", index=False)

# Cierra la conexión a la base de datos
engine.dispose()

## Verificamos que este los datos en la BD

In [36]:
connection= psycopg2.connect(**db_params)
cursor = connection.cursor()
consulta = "SELECT * FROM mi_tabla"

cursor.execute(consulta)
#Obtener los resultados de la consulta
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)
cursor.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



('Jana', 'Mraz', 'rudolph45@gmail.com', '2019-11-26', 'Peru', 10, 'Trainee', 'Development - CMS Backend', 9, 4)
('Phyllis', 'Stoltenberg', 'dayana.hagenes37@yahoo.com', '2022-04-05', 'Iceland', 5, 'Architect', 'QA Automation', 6, 6)
('Desmond', 'Crist', 'bertrand58@yahoo.com', '2019-08-23', 'Togo', 3, 'Intern', 'DevOps', 3, 8)
('Broderick', 'Lind', 'russ64@gmail.com', '2020-09-04', 'Estonia', 12, 'Lead', 'Development - FullStack', 10, 10)
('Kianna', 'Halvorson', 'dorothea.rosenbaum51@yahoo.com', '2020-01-04', 'Cape Verde', 30, 'Mid-Level', 'QA Automation', 6, 10)
('Lambert', 'Hoppe', 'issac_walker@yahoo.com', '2020-10-02', 'Saint Barthelemy', 1, 'Junior', 'Social Media Community Management', 2, 6)
('Art', 'Goyette', 'mallie27@gmail.com', '2021-03-18', 'Ghana', 27, 'Lead', 'Sales', 2, 7)
('Alycia', 'Howell', 'jayson_nikolaus@hotmail.com', '2018-12-09', 'Taiwan', 7, 'Mid-Level', 'Development - FullStack', 0, 0)
('Asia', 'Robel', 'arjun19@yahoo.com', '2018-08-13', 'Tuvalu', 24, 'Senior', 